#scikit-learn es

un paquete de Python diseñado para aplicaciones de aprendizaje automático. Junto con NumPy y pandas, es otro componente esencial en el ecosistema de ciencia de datos en Python. scikit-learn ofrece herramientas eficientes y fáciles de usar que pueden aplicarse a problemas comunes de aprendizaje automático, incluyendo análisis exploratorio y predictivo de datos.

Este cuaderno brinda una introducción a cómo Python puede aplicarse en el campo del aprendizaje automático, en particular para el análisis predictivo de datos.

###El análisis predictivo de datos

es un área del aprendizaje automático que se basa en algoritmos de clasificación y regresión. Tanto la clasificación como la regresión utilizan datos pasados para hacer predicciones sobre nuevos datos. Sin embargo, la clasificación organiza los datos en categorías discretas, mientras que la regresión puede producir un rango continuo de valores numéricos.

En este cuaderno, veremos un ejemplo de clasificación implementado con scikit-learn. Construiremos un modelo predictivo que pueda analizar reseñas de productos de clientes y clasificarlas en dos categorías: reseñas positivas y reseñas negativas. El modelo aprenderá a partir de ejemplos ya clasificados cómo predecir la clase de otros ejemplos. Una vez que hayamos entrenado el modelo, le mostraremos nuevas reseñas para que las clasifique como positivas o negativas.

###Obtención de un conjunto de datos de ejemplo

Para ser preciso, un modelo predictivo debe entrenarse con un gran número de muestras etiquetadas, por lo que el primer paso para construir un modelo que pueda clasificar reseñas de productos es obtener un conjunto de reseñas que ya estén etiquetadas como positivas o negativas. Esto nos evita tener que recopilar reseñas por nuestra cuenta y etiquetarlas manualmente.

Existen varias fuentes en línea para conjuntos de datos etiquetados. Una de las mejores es el repositorio de aprendizaje automático de la Universidad de California en Irvine, disponible en https://archive.ics.uci.edu/ml/index.php. Buscamos en el repositorio el término “reseñas de productos de clientes” y encontramos un enlace al conjunto de datos Sentiment Labelled Sentences Data Set (o alternativamente, simplemente en el navegador https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences). Descargamos y descomprimimos el archivo sentiment labelled sentences.zip desde la página web del conjunto de datos.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/content/amazon_cells_labelled.txt", names=['review', 'sentiment'], sep='\t')

In [4]:
df.head(5)

,review,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


####Dividir el conjunto de datos en un conjunto de entrenamiento y uno de prueba

Una vez importado el conjunto de datos, el siguiente paso es dividirlo en dos partes: una para entrenar el modelo predictivo y otra para probar su precisión. scikit-learn nos permite hacerlo con solo unas pocas líneas de código:

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
reviews = df['review'].values
sentiments = df['sentiment'].values
reviews_train, reviews_test, sentiment_train, sentiment_test = train_test_split(reviews, sentiments, test_size=0.2, random_state=500)

Aquí dividimos el conjunto de datos utilizando la función train_test_split() del módulo sklearn.model_selection. Las reseñas y sus correspondientes sentimientos (las puntuaciones de sentimiento) se pasan a la función como arrays de NumPy obtenidos a través de la propiedad values de los objetos Series extraídos del DataFrame.

El parámetro test_size controla cómo se divide el conjunto de datos. El valor 0.2 significa que el 20% de las reseñas se asignará aleatoriamente al conjunto de prueba. Así, seguimos el patrón 80/20; el 80% restante de las reseñas conformará el conjunto de entrenamiento. El parámetro random_state inicializa el generador de números aleatorios interno necesario para dividir los datos de forma aleatoria.

###Transformar texto en vectores de características numéricas

Para entrenar y probar nuestro modelo, necesitamos una forma de representar los datos de texto numéricamente. Aquí es donde entra el modelo de bolsa de palabras (Bag of Words o BoW). Este modelo representa un texto como el conjunto (bolsa) de sus palabras para generar datos numéricos sobre el texto. La característica numérica más típica generada por el modelo BoW es la frecuencia de palabras, es decir, el número de veces que cada palabra aparece en el texto. Este ejemplo simple muestra cómo el modelo BoW transforma un texto en un vector de características numéricas basado en la frecuencia de palabras:

    Texto: Sé que lo sabes. Tú también lo sabes.

    BoW: {"sé":1, "que":1, "lo":2, "sabes":2, "tú":1, "también":1}

    Vector: [1, 1, 2, 2, 1, 1]

Podemos usar la función CountVectorizer() de scikit-learn para crear una matriz BoW para datos de texto.

CountVectorizer() convierte datos de texto en vectores de características numéricas (vectores n-dimensionales de características numéricas que representan algún objeto) y realiza la tokenización (separar un texto en palabras individuales y símbolos de puntuación) utilizando el tokenizador predeterminado o uno personalizado. Un tokenizador personalizado puede implementarse con una herramienta de procesamiento de lenguaje natural como spaCy. En este ejemplo, usaremos la opción predeterminada para simplificar.

Así es como convertimos las reseñas en vectores de características:

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(reviews)
X_train = vectorizer.transform(reviews_train)
X_test = vectorizer.transform(reviews_test)

Primero, creas un objeto vectorizador. Luego, aplicas el método fit() del vectorizador para construir el vocabulario de tokens encontrado en el conjunto de datos de reseñas, que contiene todas las reseñas de los conjuntos de entrenamiento y prueba. Después, utilizas el método transform() del objeto vectorizador para transformar los datos de texto en los conjuntos de entrenamiento y prueba en vectores de características numéricas.

###Entrenar y evaluar el modelo

Ahora que tenemos los conjuntos de entrenamiento y prueba en forma de vectores numéricos, estamos listo/tas para entrenar y probar el modelo.

Primero, entrenaremos el clasificador LogisticRegression() de scikit-learn para predecir el sentimiento de una reseña. La regresión logística es un algoritmo básico pero popular para resolver problemas de clasificación.

Aquí, creamos un clasificador LogisticRegression() y luego usamos su método fit() para entrenar el modelo según los datos de entrenamiento proporcionados:

In [8]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, sentiment_train)

LogisticRegression()

Ahora debemos evaluar qué tan bien el modelo puede hacer predicciones con nuevos datos. Para esto, necesitamos un conjunto de datos etiquetado, por lo que es común dividir un conjunto de datos etiquetado en un conjunto de entrenamiento y uno de prueba, como hicimos antes.
Acá evaluamos el modelo utilizando el conjunto de prueba:

In [9]:
accuracy = classifier.score(X_test, sentiment_test)
print("Accuracy:", accuracy)

Accuracy: 0.81


La precisión generalmente aparece de la siguiente manera:

    Precisión: 0.81

Esto significa que el modelo tiene una precisión del 81%. Si experimentas con el parámetro random_state en la función train_test_split(), puedes obtener un valor ligeramente diferente debido a que las instancias para los conjuntos de entrenamiento y prueba se seleccionan aleatoriamente del conjunto original.

###Realizar predicciones con datos nuevos

Ahora que entrenamos y probamos nuestro modelo, está listo para analizar nuevos datos sin etiquetar. Esto nos dará una imagen más completa de qué tan bien está funcionando el modelo.
Probemoslo alimentando al modelo algunas reseñas nuevas de ejemplo:

In [11]:
new_reviews = ['bad version of Windows','one star, but nice job!', 'Old version of python but good for my work', 'Very good effort, but not five stars', 'Clear and concise']
X_new = vectorizer.transform(new_reviews)
print(classifier.predict(X_new))

[0 1 1 0 1]


Empezamos creando una lista de nuevas reseñas, luego transformamos ese nuevo texto en vectores de características numéricas. Finalmente, predecimos los sentimientos de clase para las nuevas muestras. Los sentimientos se devuelven como una lista:

    [0, 1, 1]

Recorda:

    0 indica una reseña negativa
    1 indica una reseña positiva.
    
Como podemos ver, el modelo funcionó para estas reseñas de ejemplo, mostrando que la primera es negativa y las otras dos son positivas.

###Resumen

Vimos cómo acceder y manipular datos. Tambien, utilizamos scikit-learn, una popular biblioteca de aprendizaje automático de Python, para construir un modelo predictivo para clasificación.

